In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

In [2]:
# read next fast speed
dataset = load_from_disk("../processed_datadir/wikitext-103-story-train")

In [3]:
print(dataset.features)
print(len(dataset))

{'text': Value(dtype='string', id=None)}
29525


In [ ]:
for i in range(0, 3):
    print(dataset[i])

In [ ]:
for i in range(0,100):
    print(len(dataset[i]["text"]))

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(f"bert-base-uncased")

In [5]:
tokenizer.model_max_length = 4096
tokenizer.save_pretrained("../tokenizer_save/tokenizer-bert-base-uncased-4096/")

('../tokenizer_save/tokenizer-bert-base-uncased-4096/tokenizer_config.json',
 '../tokenizer_save/tokenizer-bert-base-uncased-4096/special_tokens_map.json',
 '../tokenizer_save/tokenizer-bert-base-uncased-4096/vocab.txt',
 '../tokenizer_save/tokenizer-bert-base-uncased-4096/added_tokens.json',
 '../tokenizer_save/tokenizer-bert-base-uncased-4096/tokenizer.json')

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../tokenizer_save/tokenizer-bert-base-uncased-4096/")

In [7]:
import multiprocessing

num_proc = multiprocessing.cpu_count() - 4
print(num_proc)
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    # print(len(examples["text"]))
    tokenized_inputs = tokenizer(examples["text"], truncation=True,   # 在这就不带文本了
        max_length=tokenizer.model_max_length, return_overflowing_tokens=True,return_length=True)
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []


    
    for length, input_ids in zip(tokenized_inputs["length"], tokenized_inputs["input_ids"]):
        if length == tokenizer.model_max_length:
            input_ids_list.append(input_ids)
        else:
            input_ids_list.append(input_ids + [tokenizer.pad_token_id] * (tokenizer.model_max_length - length))
    for length, token_type_ids in zip(tokenized_inputs["length"], tokenized_inputs["token_type_ids"]):
        if length == tokenizer.model_max_length:
            token_type_ids_list.append(token_type_ids)
        else:
            token_type_ids_list.append(token_type_ids + [0] * (tokenizer.model_max_length - length))
    for length, attention_mask in zip(tokenized_inputs["length"], tokenized_inputs["attention_mask"]):
        if length == tokenizer.model_max_length:
            attention_mask_list.append(attention_mask)
        else:
            attention_mask_list.append(attention_mask + [0] * (tokenizer.model_max_length - length))
    print("success")
    return {"input_ids": input_ids_list, "token_type_ids": token_type_ids_list, "attention_mask": attention_mask_list}

# preprocess dataset
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features

12
The max length for the tokenizer is: 4096


Map (num_proc=12):   0%|          | 0/29525 [00:00<?, ? examples/s]

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
for i in range(0, len(tokenized_datasets)):
    # print(tokenized_datasets[i])
    for k in tokenized_datasets[i]:
        # print(k)
        # print(len(tokenized_datasets[i][k]))
        if len(tokenized_datasets[i][k]) != tokenizer.model_max_length:
            print("error")
            

In [9]:
# Split preprocessed dataset into train, validation, and test sets
splits = tokenized_datasets.train_test_split(test_size=0.03)
preprocessed_splits = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_from_disk("../processed_datadir/wikitext-103-story-test/")
})

In [10]:
preprocessed_splits.save_to_disk("../processed_datadir/wikitext-103-story-bert-4096")

Flattening the indices:   0%|          | 0/41928 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/41928 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1297 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1297 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/62 [00:00<?, ? examples/s]

In [11]:
print(len(preprocessed_splits["train"]))

41928
